---
title: Another take on LU-Factorization
subject:  Linear Algebraic Systems
subtitle: Peeling the Onion
short_title: Peeling the LU onion
authors:
  - name: Nikolai Matni
    affiliations:
      - Dept. of Electrical and Systems Engineering
      - University of Pennsylvania
    email: nmatni@seas.upenn.edu
license: CC-BY-4.0
keywords: Gaussian Elimination, LU factorization
math:
  '\vv': '\mathbf{#1}'
  '\bm': '\begin{bmatrix}'
  '\em': '\end{bmatrix}'
  '\R': '\mathbb{R}'
---

This section is based on Ch 5.3 of Jessy Grizzles [ROB 101 notes](https://github.com/michiganrobotics/rob101/blob/main/Fall%202021/Textbook/ROB_101_December_2021_Grizzle.pdf).

## Learning Objectives

By the end of this page, you should know:
- another way of computing the $LU$ factorization of a matrix

## LU (Lower-Upper) Factorization: Regular Case
In the previous section, we saw a correct and perfectly acceptable way of solving systems of linear equations with regular $A$ matrices.  It turns out that it is very closely related to an approach based on _factorizing_ the coefficient matrix $A$ into a product of a lower triangular matrix $L$ and and upper triangular matrix $U$ such that $A=LU$, which we'll develop in this section. You'll see that the expressions end up being somewhat simpler to work with, and that the pseudocode is a bit "cleaner" (specifically, we won't have nested for loops).  This is also _much_ closer to the way solutions to linear systems are implemented in modern linear algebra computational packages.

We'll start with a worked example, and then define the general algorithm.  

### Column-Row Multiplication
A special case of matrix multiplication is multiplying a $m \times 1$ column vector $\vv c$ and a $1 \times n$ row vector $\vv r$ together.  We'll work with $m=n=3$ here, but the general case is very similar.  Applying the rules of matrix arithmetic, we see that if
$$
\vv c = \bm c_1\\ c_2\\ c_3\em, \quad \vv r = \bm r_1 &  r_2 & r_3\em,$$
then
$$\vv c \vv r = \bm r_1 \vv c & r_2 \vv c & r_3 \vv c\em = \bm c_1r_1 & c_1 r_2 & c_1 r_3 \\c_2r_1 & c_2 r_2 & c_2 r_3\\ c_3r_1 & c_3 r_2 & c_3 r_3\em = \bm c_1 \vv r \\ c_2 \vv r \\ c_3 \vv r\em.
$$
(prop1)=
For this section, the most important property we will use is that the $i$th row of $\vv c \vv r$, given by $c_i \vv r$, is a copy of the row vector $\vv r$ scaled by the corresponding component $c_i$ of the column vector $\vv c$.

### Pealing the onion
Consider the square matrix
$$
A = \bm 1 & 4 & 5 \\ 2 & 9 & 17 \\ 3 & 18 & 58 \em.
$$
Our goal is to find a a column vector $\vv c_1$ and a row vector $\vv r_1$ such that
$$
A - \vv c_1 \vv r_1 = \bm 0 & 0 & 0\\ 0 & \star & \star \\ 0 & \star & \star\em,
$$
where here $\star$ means we do not care about the specific values.  Another way of saying this is that we want to zero out the first row and column of $M$ by choosing $\vv c_1$ and $\vv r_1$ so that the first row and first column of the matrix product $\vv c_1 \vv r_1$ match those of $M$.  Can we do this?

In the special case when the $(1,1)$ entry of $A$ is equal to 1, i.e., when $a_{11}=1$, we can do this pretty easily!  We'll do the obvious thing and just define $\vv c_1$ and $\vv r_1$ to be the first column of $M$ and the first row of $M$, respectively, that is, $\vv c_1 = (1,2,3)$ and $\vv r_1 = \bm 1 & 4 & 5\em$.[^brackets]  Then, remembering the [property](#prop1) that we identified earlier, we have that
$$
\vv c_1 \vv r_1 = \bm 1 \\ 2 \\ 3\em \bm 1 & 4 & 5\em=\bm 1 & 4 & 5 \\ 2 & 8 & 10 \\ 3 & 12 & 15\em,
$$
and would you look at that, we met our objective:
$$
A = \bm \underline 1 & \underline 4 & \underline 5 \\ \underline 2 & 9 & 17 \\ \underline 3 & 18 & 58 \em, \quad \vv c_1 \vv r_1 = \bm \underline 1 & \underline 4 & \underline 5 \\ \underline 2 & 8 & 10 \\ \underline 3 & 12 & 15\em.
$$
We can then compute
\begin{eqnarray}
A - \vv c_1 \vv r_1 &= \bm \underline 1 & \underline 4 & \underline 5 \\ \underline 2 & 9 & 17 \\ \underline 3 & 18 & 58 \em - \bm \underline 1 & \underline 4 & \underline 5 \\ \underline 2 & 8 & 10 \\ \underline 3 & 12 & 15\em\\
& = \bm \underline 0 & \underline 0 & \underline 0 \\ \underline 0 & 1 & 7 \\ \underline 0 & 6 & 43\em.
\end{eqnarray}

[^brackets]: Remember that $\vv c_1 = (1,2,3)$ is a $3 \times 1$ column vector because of the round brackets and commas, and $\vv r_1 = \bm 1 & 4 & 5\em$ is a $1 \times 3$ row vector because of the square brackets and no commas.

In doing this, we've taken a $3 \times 3$ matrix and essentially turned it into a $2 \times 2$ matrix.  Let's see if we can do it again: define $\vv c_2$ and $\vv r_2$ to be the second column and second row of $A-\vv c_1\vv r_1$:
$$
\vv c_2 = \bm 0 \\ 1 \\ 6 \em, \quad \vv r_2 = \bm 0 & 1 & 7\em.
$$
We then compute
$$
\vv c_2 \vv r_2 = \bm 0 \\ 1 \\ 6 \em\bm 0 & 1 & 7\em=\bm 0 & 0 & 0 \\ 0 & 1 & 7 \\ 0 & 6 & 42\em,
$$
to obtain
$$
A- \vv c_1 \vv r_1 = \bm  0 & 0 & 0 \\ 0 & \underline 1 & \underline 7 \\ 0 & \underline 6 & 43\em, \quad \vv c_2 \vv r_2 = \bm  0 & 0 & 0 \\ 0 & \underline 1 & \underline 7 \\ 0 & \underline 6 & 42\em.
$$
Next we subtract the latter from the former:
\begin{eqnarray}
(A - \vv c_1\vv r_1) - \vv c_2 \vv r_2 &= \bm  0 & 0 & 0 \\ 0 & \underline 1 & \underline 7 \\ 0 & \underline 6 & 43\em - \bm  0 & 0 & 0 \\ 0 & \underline 1 & \underline 7 \\ 0 & \underline 6 & 42\em \\
& = \bm  \underline 0 & \underline 0 & \underline 0 \\ \underline 0 & \underline 0 & \underline 0 \\ \underline 0 & \underline 0 & 1\em.
\end{eqnarray}

Just like that, we're down to what is essentially a $1 \times 1$ matrix.  We'll quickly note that $\vv c_3 = (0, 0, 1)$ and $\vv r_3 = \bm 0 & 0 & 1\em$ satisfies
$$
\vv c_3 \vv r_3 =  \bm  \underline 0 & \underline 0 & \underline 0 \\ \underline 0 & \underline 0 & \underline 0 \\ \underline 0 & \underline 0 & 1\em,
$$
and we then immediately have that $A - \vv c_1 \vv r_1 - \vv c_2 \vv r_2 -\vv c_3 \vv r_3 = 0$, or equivalently
\begin{equation}
A = \vv c_1 \vv r_1 + \vv c_2 \vv r_2 + \vv c_3 \vv r_3 =\underbrace{\bm \vv c_1 & \vv c_2 & \vv c_3\em}_{L}\underbrace{\bm\vv r_1 \\ \vv r_2 \\ \vv r_3\em}_{U}.
\end{equation}
Here we used the properties of [block matrix multiplication](./022-linsys-matvec.ipynb#blockmatmul) we saw last time.

In the above, used $L$ and $U$ for two special matrices that were built up from the $\vv c_i$ and $\vv r_i$ we have identified so far:
- $L = \bm \vv c_1 & \vv c_2 & \vv c_3\em = \bm 1 & 0 & 0 \\ 2 & 1 & 0 \\ 3 & 6 & 1\em$  is _lower triangular_,
- $U = \bm\vv r_1 \\ \vv r_2 \\ \vv r_3\em = \bm 1 & 4 & 5 \\ 0 & 1 & 7 \\ 0 & 0 & 1 \em$ is _upper triangular_, and
- $A=LU$, that is we _factored_ our original matrix $M$ into a product of a lower triangular matrix $L$ and an upper triangular matrix $U$.

### A More General Version
We got lucky in the previous example: at every step of the way, the upper left most entry of the matrix was equal to 1.  In general, this won't be the case, but we'll see that for regular matrices $A$, our trusty friend the _pivot_ will help us out.  Again, we will start with a simple example to work out the main idea, and then state a more general principle.  The key fact we use throughout is that for a regular matrix, _all of the pivots are nonzero._

Let's try to compute an $LU$-factorization for the matrix 
$$
A = \bm 2 & 3 \\ 4 & 5 \em.
$$

If we use our prior strategy of setting $\vv c_1 = (2, 4)$ to be the first column of $M$, and $\vv r_1 = \bm 2 & 3\em$ to be the first row of $M$, we run into trouble, because
$$
A - \vv c_1 \vv r_1 = \bm 2 & 3 \\ 4 & 5 \em - \bm 2 \\ 4 \em\bm 2 & 3\em=\bm 2 & 3 \\ 4 & 5 \em -\bm 4 & 6 \\ 8 & 12 \em = \bm -2 & -3 \\ -4 & -7\em.
$$
This sadly does **not** zero out the first row and column of $M$.  The secret here is to build $\vv c_1$ in a slightly different way: we will instead set $\vv c_1$ to be the first column of $A$ divided by $a_{11}$.  Remember, $a_{11}$ is the _first pivot_ of $A$.  So keeping $\vv r_1$ the same, and instead setting $\vv c_1 = (\frac{2}{2}, \frac{4}{2})=(1,2)$, we then get exactly what we were hoping for:
$$
A - \vv c_1 \vv r_1 = \bm 2 & 3 \\ 4 & 5 \em - \bm \frac{2}{2} \\ \frac{4}{2} \em\bm 2 & 3\em=\bm 2 & 3 \\ 4 & 5 \em -\bm 2 & 3 \\ 4 & 6 \em = \bm 0 & 0\\ 0 & -1\em.
$$

More generally, here is the basic recipe for _peeling the onion_ when matrix pivots are not equal to one (but are also nonzero!):
\begin{eqnarray}
\bm 
a_{11} & a_{12} & \cdots & a_{1n} \\
a_{21} & a_{22} & \cdots & a_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
a_{n1} & a_{n2} & \cdots & a_{nn}
\em
-\frac{1}{a_{11}}\bm a_{11} \\ a_{21} \\ \vdots  \\ a_{n1}\em \bm a_{11} & a_{12} & \cdots & a_{1n} \em \\=
\bm 
a_{11} & a_{12} & \cdots & a_{1n} \\
a_{21} & a_{22} & \cdots & a_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
a_{n1} & a_{n2} & \cdots & a_{nn}
\em - \bm 
a_{11} & a_{12} & \cdots & a_{1n} \\
a_{21} & \star & \cdots & \star \\
\vdots & \vdots & \ddots & \vdots \\
a_{n1} & \star & \cdots & \star
\em \\ = \bm 
0 & 0 & \cdots & 0 \\
0 & \star & \cdots & \star \\
\vdots & \vdots & \ddots & \vdots \\
0 & \star & \cdots & \star
\em
\end{eqnarray}

## Optional: $LU$-factorization is Gaussian Elimination!
TODO: make this clear.

We have $A = LU$.  Suppose we have applied Gaussian Elimination to get our system into $U\vv x = \vv c$.  There is a matrix $M$ such that $MA= U$ and $M\vv b=\vv c$.  So we have 
$$
MA\vv x = MLU\vv x = M\vv b
$$
But if $MA = U$ then we must have that $ML= I$.

Now, if we go back to solving our system using $LU$-factorization, we have that $LU\vv x = \vv b$ which we can turn into $L\vv z = \vv b$ an $U\vv x = \vv z$.  We saw that we can solve that $L\vv z = b$ by forward substitution, but notice that we also have that $ML\vv z = \vv z = M \vv b$, i.e., the matrix $M$ that reduces our original system of equations to upper triangular form is actually performing this forward substitution.